In [3]:
import numpy as np
import pandas as pd
import json

In [4]:
#Load the diseases master json dataset from ChaptersCleaned folder

with open('ChaptersCleaned/everything_about_dogs_diseases_master.json', 'r') as file:
    diseases_master = json.load(file)

In [7]:
#Use embedding model from sentence_transformers
from sentence_transformers import SentenceTransformer
# Load the model
model = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\Akshay\Documents\VetRAG\VetRAG_Dataset\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Akshay\Documents\VetRAG\VetRAG_Dataset\.venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Akshay\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run

In [8]:
#Save the model to reuse later
model.save('diseases_embedding_model')

In [6]:
#Iterate through each disease, convert the disease name along with all values to string and make embeddings
char_counts = []
for disease in diseases_master:
    # Convert the disease name and values to string
    disease_name = str(disease)
    disease_values = str(diseases_master[disease])
    
    # Create a combined string for embedding
    combined_string = f"{disease_name} {disease_values}"
    print("Approximate character count for embedding:", len(combined_string))
    char_counts.append(len(combined_string))
    

Approximate character count for embedding: 17221
Approximate character count for embedding: 8930
Approximate character count for embedding: 11711
Approximate character count for embedding: 11612
Approximate character count for embedding: 9506
Approximate character count for embedding: 10339
Approximate character count for embedding: 12720
Approximate character count for embedding: 10031
Approximate character count for embedding: 9345
Approximate character count for embedding: 11757
Approximate character count for embedding: 16331
Approximate character count for embedding: 14014
Approximate character count for embedding: 10173
Approximate character count for embedding: 9753
Approximate character count for embedding: 8992
Approximate character count for embedding: 8882
Approximate character count for embedding: 10677
Approximate character count for embedding: 9301
Approximate character count for embedding: 14658
Approximate character count for embedding: 9500
Approximate character count 

In [9]:
import numpy as np
min(char_counts), max(char_counts), np.mean(char_counts), np.std(char_counts)

(8682, 32645, np.float64(10962.153846153846), np.float64(2780.151188730895))

In [9]:
#Iterate through each disease, convert the disease name along with all values to string and make embeddings

for disease in diseases_master:
    # Convert the disease name and values to string
    disease_name = str(disease)
    disease_values = str(diseases_master[disease])
    
    # Create a combined string for embedding
    combined_string = f"{disease_name} {disease_values}"
    print("Creating embedding for disease:", disease_name)
    # Generate the embedding
    embedding = model.encode(combined_string)
    
    # Store the embedding in the dictionary
    diseases_master[disease]['embedding'] = embedding.tolist()  # Convert numpy array to list for JSON serialization

Creating embedding for disease: Asthma


c:\Users\Akshay\Documents\VetRAG\VetRAG_Dataset\.venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Creating embedding for disease: Apoplexy
Creating embedding for disease: Abscess
Creating embedding for disease: Abortion
Creating embedding for disease: Asphyxia
Creating embedding for disease: Bladder Inflammation of the
Creating embedding for disease: Bladder Paralysis
Creating embedding for disease: BOILS
Creating embedding for disease: Blood Poverty of the
Creating embedding for disease: Bites
Creating embedding for disease: Bowels Inflammation of Enteritis Peritonitis
Creating embedding for disease: Colic comes on quickly and is relieved by rubbing the stomach
Creating embedding for disease: Bad Breath
Creating embedding for disease: Balanatis
Creating embedding for disease: Liniment for Sprains Bruises Etc
Creating embedding for disease: Burns and Scalds
Creating embedding for disease: Burns
Creating embedding for disease: Baldness
Creating embedding for disease: Blain
Creating embedding for disease: Brain Inflammation of the Meningitis
Creating embedding for disease: Blisters
C

In [10]:
#Now convert the diseases_master dictionary to a DataFrame
diseases_df = pd.DataFrame.from_dict(diseases_master, orient='index')

In [13]:
#Convert the diseases_df to parquet format
#diseases_df.to_parquet('diseases_embeddings.parquet')
#Also save the diseases_master dictionary to a JSON file
with open('diseases_master_with_embeddings.json', 'w') as file:
    json.dump(diseases_master, file, indent=4)

In [ ]:
#Validate the embeddings by running a retrieval task
prompt = ""


# Generate the embedding for the prompt
prompt_embedding = model.encode(prompt)
# Convert the prompt embedding to a numpy array
prompt_embedding = np.array(prompt_embedding).reshape(1, -1)
# Find the most similar disease based on the embedding
from sklearn.metrics.pairwise import cosine_similarity
similarities = []
for disease, data in diseases_master.items():
    if 'embedding' in data:
        disease_embedding = data['embedding']
        #Convert the string list back to numpy array
        disease_embedding = np.array(disease_embedding).reshape(1, -1)
        similarity = cosine_similarity(prompt_embedding,disease_embedding)
        similarities.append((disease, similarity[0][0]))

# Sort the diseases based on similarity
similarities.sort(key=lambda x: x[1], reverse=True)

c:\Users\Akshay\Documents\VetRAG\VetRAG_Dataset\.venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [25]:
similarities

[('Dyspepsia', np.float64(0.4553810249115964)),
 ('Looseness of the Bowels', np.float64(0.4122600993220886)),
 ('Biliousness', np.float64(0.3931982915110197)),
 ('Indigestion Dyspepsia', np.float64(0.3906972532390177)),
 ('Stomach Inflammation of Gastritis', np.float64(0.3793625854946379)),
 ('Enteritis', np.float64(0.37809955500280723)),
 ('Cantharides Spanish Fly', np.float64(0.36815132653131977)),
 ('Tonic Stomachic Pills', np.float64(0.36736212779231525)),
 ('Stomach Inflammation of', np.float64(0.3636829099030592)),
 ('Cold in the Head', np.float64(0.36097739123894274)),
 ('St Vitus Dance', np.float64(0.35894329425733856)),
 ('Brain Inflammation of the Meningitis', np.float64(0.35824293565957765)),
 ('Purging', np.float64(0.35361562639174726)),
 ('Coryza', np.float64(0.34479686119892416)),
 ('Hernia Umbilical', np.float64(0.3438357105655461)),
 ('Gastritis', np.float64(0.34321289277467437)),
 ('Keeping Flies Off Ear and Nose', np.float64(0.33522781210975483)),
 ('Meningitis', np.f

In [1]:
#Store the disease_master in parquet format
#Load the json
import json
with open('diseases_master_with_embeddings.json', 'r') as file:
    diseases_master = json.load(file)

In [2]:
#Convert the json to parquet
import pandas as pd
diseases_df = pd.DataFrame.from_dict(diseases_master, orient='index')


In [11]:
#Save as parquet
diseases_df.to_parquet('diseases_dataset_with_embeddings.parquet')

In [ ]:
#Also 